<a href="https://colab.research.google.com/github/ueki5/colaboratory/blob/main/playground-018.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip install torchviz | tail -n 1
%pip install torchinfo | tail -n 1
%pip install torchvision | tail -n 1
%pip install japanize-matplotlib | tail -n 1
%pip install plotly | tail -n 1

In [ ]:
import numpy as np
import torch
import matplotlib.pyplot as plt
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import japanize_matplotlib
japanize_matplotlib.japanize()
from torch.utils.data import DataLoader

# デバイスの割り当て
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

# ダウンロード先ディレクトリ名
data_root = './data'

# # ReLU関数
# relu = torch.nn.ReLU()
# x_np = np.arange(-2, 2.1, 0.25)
# x = torch.tensor(x_np).float()
# y = relu(x)

# plt.plot(x.data, y.data)
# plt.title('ReLU関数')
# plt.show()

# # テスト用テンソル変数x, y
# x_np = np.arange(-2, 2.1, 0.25)
# y_np = np.arange(-1, 3.1, 0.25)
# x = torch.tensor(x_np).float()
# y = torch.tensor(y_np).float()

# # 変数xをGPUに送る
# x = x.to(device)
# y = y.to(device)

# # 変数xとyのdevice属性の確認
# print('x:', x.device)
# print('y:', y.device)

# # xとyの間の演算
# z = x * y
# print(z)

# train_set0 = datasets.MNIST(
#     # 元データダウンロード先の指定
#     root = data_root,
#     # 訓練データか検証データか
#     train = True,
#     # 元データがない場合にダウンロードするか
#     download = True)

# # ダウンロードしたファイルの確認
# !ls -lR ./data/MNIST

# # データ件数の確認
# print('train_set0.data.shape：', train_set0.data.shape)
# print('データ件数：', len(train_set0))

# # 最初の要素の取得
# image, label = train_set0[0]

# # データ型の確認
# print('入力データの型：', type(image))
# print('正解データの型：', type(label))
# print(f'image.format:{image.format}')
# print(f'image.size:{image.size}')
# print(f'image.mode:{image.mode}')
# print(f'image.getextrema:{image.getextrema()}')
# for i in range(image.size[0]):
#   for j in range(image.size[1]):
#     print(f'image.getpixel(({i},{j})):{image.getpixel((i, j))}')

# # 入力データの画像表示
# plt.figure(figsize=(1, 1))
# plt.title(f'{label}')
# plt.imshow(image, cmap='gray_r')
# plt.axis('off')
# plt.show()

# # 正解データ付きで、最初の２０個をイメージ表示
# plt.figure(figsize=(10, 3))
# for i in range(20):
#   ax = plt.subplot(2, 10, i + 1)

#   # imageとlabelの所得
#   image, label = train_set0[i]

#   # イメージ表示
#   plt.imshow(image, cmap='gray_r')
#   ax.set_title(f'{label}')
#   ax.get_xaxis().set_visible(False)
#   ax.get_yaxis().set_visible(False)
# plt.show()

# テンソル化 ＋ 正規化 ＋ １階テンソル化
transform = transforms.Compose([
  # データのテンソル化
  transforms.ToTensor(),
  # データの正規化(Normalize(μ, σ) ⇒ (x - μ) / σ)
  transforms.Normalize(0.5, 0.5),
  # １階テンソル化
  transforms.Lambda(lambda x: x.view(-1)),
])

# 訓練データ
train_set = datasets.MNIST(
  root=data_root,
  train=True,
  download=True,
  transform=transform
)

# 検証データ
test_set = datasets.MNIST(
  root=data_root,
  train=False,
  download=True,
  transform=transform
)
# 変換結果の確認
image, label = train_set[0]
print('訓練データの型：', type(image))
print('訓練データのshape：', image.shape)
print('最小値：', image.data.min())
print('最大値：', image.data.max())

image, label = test_set[0]
print('検証データの型：', type(image))
print('検証データのshape：', image.shape)
print('最小値：', image.data.min())
print('最大値：', image.data.max())

# ミニバッチのサイズ指定
batch_size = 500

# 訓練用データローダー
# 訓練用なので、シャッフルをかける
train_loader = DataLoader(
    train_set,
    batch_size = batch_size,
    shuffle = True,
)

# 検証用データローダー
# 検証用にシャッフルは不要
test_loader = DataLoader(
    test_set,
    batch_size = batch_size,
    shuffle = False,
)

# 何組のデータが取得できるか？
print(len(train_loader))

# データローダーから最初の１セットを取得する
for images, lables in train_loader:
    break

print('images.shape:', images.shape)
print('lables.shape:', lables.shape)

# 入力次元数
n_input = images.shape[0]

# 出力次元数
n_output = 10

# 隠れ層の次元数
n_hidden = 128

# 結果確認
print(f'n_input: {n_input}, n_hidden: {n_hidden}, n_output: {n_output}')

###################################################
# 関数定義
###################################################
lr = 0.01  # 学習率
num_epochs = 100 # 繰り返し回数
num_history = 10 # 履歴採取タイミング
class Net(torch.Module):
    def __init(self, n_input, n_output, n_hidden):
        super().__init()

        # 入力層の定義
        self.l1 = torch.nn.Linear(n_input, n_hidden)

        # 隠れ層の定義
        self.l2 = torch.nn.Linear(n_hidden, n_output)

        # ReLU関数
        self.relu = torch.nn.ReLU(inplace=True)

        # 関数合成
        self.net = torch.nn.Sequential(
            self.l1,
            self.relu,
            self.l2,
        )

    # 予測関数    
    def forward(self, inputs):
      outputs = self.net(inputs)
      return outputs

# 予測計算オブジェクトの作成
net = Net(n_input, n_output, n_hidden)

# 損失関数
criterion = torch.nn.CrossEntropyLoss()

# 最適化関数
optimizer = torch.optim.SGD(net.parameters(), lr=lr)

# 記録用配列初期化
history = np.zeros((0, 5))


cpu
訓練データの型： <class 'torch.Tensor'>
訓練データのshape： torch.Size([784])
最小値： tensor(-1.)
最大値： tensor(1.)
検証データの型： <class 'torch.Tensor'>
検証データのshape： torch.Size([784])
最小値： tensor(-1.)
最大値： tensor(1.)
120
images.shape: torch.Size([500, 784])
lables.shape: torch.Size([500])
